## make_bry:
### Make lateral boundary conditions, using salt and temp, u, v, zeta, from BRAN
#### Version id:
#### v1.

In [1]:
cd /g/data/jk72/deg581/se-qld-setup/notebooks

/g/data/jk72/deg581/se-qld-setup/notebooks


In [14]:
run = 'seqld_1km'

import xarray as xr
from netCDF4 import Dataset
import numpy as np 
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import CloughTocher2DInterpolator
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import datetime
import glob

proj_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir = os.path.join(proj_dir,'data')
src_dir = os.path.join(proj_dir,'src')

sys.path.append(src_dir)

from ext.tools.calc_z import calc_z as calc_z_old
try: 
    from functions import GridStiffness
except:
    print('couldn''t load')

from functions import GridStiffness


# set user parameter and call main routine
grd_file = os.path.join(data_dir,'proc','seqld_1km_grd.nc')


import socket
comp_name = socket.gethostname()

if comp_name=='SEES-3PV4VV3':
    frc_path = os.path.join('../data/raw/BRAN2020/')
    frc_sub_paths = ['./']
else:
    frc_path = os.path.join('/g/data/gb6/BRAN/BRAN2020/')
    frc_sub_paths = ['./daily/']



# Grid parameters; check grid_file and *.in file to make sure these are correct
N=31
Vtransform=2
Vstretching=4
theta_s=1
theta_b=4
Tcline=100
hc=Tcline # if Vtransform==2, hc=Tcline

# # southernmost index of ECCO2 grid to read (1-based)
# nbdry_ecco = 300

# upper and lower bounds for temp and salinity
tempUp = 10
tempLow = -3

saltUp = 34.8
saltLow = 33.2

In [3]:
# define inpaint_nans
from scipy.signal import convolve2d
def inpaint_nans(inField):
    im = inField.copy()
    import scipy
    ipn_kernel = np.array([[1,1,1],[1,0,1],[1,1,1]]) # kernel for inpaint_nans
    nans = np.isnan(im)
    while np.sum(nans)>0:
        im[nans] = 0
        vNeighbors = convolve2d((nans==False),ipn_kernel,mode='same',boundary='symm')
        im2 = convolve2d(im,ipn_kernel,mode='same',boundary='symm')
        im2[vNeighbors>0] = im2[vNeighbors>0]/vNeighbors[vNeighbors>0]
        im2[vNeighbors==0] = np.nan
        im2[(nans==False)] = im[(nans==False)]
        im = im2
        nans = np.isnan(im)
    return im

In [4]:
#define 1d inpaint_nans
from scipy.interpolate import interp1d
def fill_nans_scipy1(padata, pkind='linear'):
    """
    Interpolates data to fill nan values

    Parameters:
        padata : nd array 
            source data with np.NaN values

    Returns:
        nd array 
            resulting data with interpolated values instead of nans
    """
    aindexes = np.arange(padata.shape[0])
    agood_indexes, = np.where(np.isfinite(padata))
    f = interp1d(agood_indexes
               , padata[agood_indexes]
               , bounds_error=False
               , copy=False
               , fill_value="extrapolate"
               , kind=pkind)
    return f(aindexes)

In [5]:
os.path.join(os.getcwd(), os.pardir)

'/g/data/jk72/deg581/se-qld-setup/notebooks/..'

In [6]:
grd = xr.open_dataset(grd_file)
grd.lon_rho.min().values, grd.lon_rho.max().values,grd.lat_rho.min().values, grd.lat_rho.max().values

grd_lon_ax = np.array((grd.lon_rho.min().values*0.9, grd.lon_rho.max().values*1.1))
grd_lat_ax = np.array((grd.lat_rho.min().values*1.1, grd.lat_rho.max().values*0.9))

In [7]:
# make the interpolation surfaces for the input grid. First use the old style indexing method
z_rho_3d, sc_r, Cs_r = calc_z_old(grd.h, np.zeros(grd.h.shape), theta_s, theta_b, hc, N)
lon_rho_3d = np.tile(grd.lon_rho, (N,1,1))
lat_rho_3d = np.tile(grd.lat_rho, (N,1,1))
lon_rho_2d = np.tile(grd.lon_rho, (1,1))
lat_rho_2d = np.tile(grd.lat_rho, (1,1))
# join together L,T,R,B
# don't flip the R,B faces as ROMS expects to reads from the i/j=1 position outwards.
DepthsInterpSurface = np.concatenate((z_rho_3d[:,:,0],z_rho_3d[:,-1,:],z_rho_3d[:,:,-1],z_rho_3d[:,0,:]),axis=1)
LatInterpSurface = np.concatenate((lat_rho_3d[:,:,0],lat_rho_3d[:,-1,:],lat_rho_3d[:,:,-1],lat_rho_3d[:,0,:]),axis=1)
LonInterpSurface = np.concatenate((lon_rho_3d[:,:,0],lon_rho_3d[:,-1,:],lon_rho_3d[:,:,-1],lon_rho_3d[:,0,:]),axis=1)

LatInterpSurface2d = np.concatenate((lat_rho_2d[:,0],lat_rho_2d[-1,:],lat_rho_2d[:,-1],lat_rho_2d[0,:]),axis=0)
LonInterpSurface2d = np.concatenate((lon_rho_2d[:,0],lon_rho_2d[-1,:],lon_rho_2d[:,-1],lon_rho_2d[0,:]),axis=0) 

In [8]:
if LonInterpSurface.min() < 0:
    print('WARNING: VALUES BELOW 0, need to wrap longitude around by +360')

In [9]:
# settings for making the netcdf file:
year_range=np.arange(2012,2013,1)
print('making data for years ',year_range[0],' to ',year_range[-1])


making data for years  2012  to  2012


In [16]:
%%time

for year in year_range:
    print('making forcing file for year ',year)

    ##/ NOT REQUIRED WITH BRAN    
    # strange mapping of access data:
    # 2007-01 -> 196, 2007-04 -> 197, ..., 2007-10 -> 199
    # mapping is a linear mapping  with slope 4
    # output = output_start + slope*(input-input_start)
    # output = 0 + 4*(input-1958)
    # output_ref1 = 4*(year-1958)
    # access_sub_paths = [('./output'+str(output_ref1)+'/ocean/'),('./output'+str(output_ref1+1)+'/ocean/'),('./output'+str(output_ref1+2)+'/ocean/'),('./output'+str(output_ref1+3)+'/ocean/')]
    ##/
    
    print('loading data from ',frc_sub_paths)
    year = str(year)
    # Master cell that runs all of the processing.
    print('set settings for each file')
    out_file = os.path.join(data_dir,'proc',run+'_daily_'+year+'_bry.nc')
    print('out file name is: ',out_file)
   

    print('start loading data')
    file_list=[]
    file_name_format = 'ocean_temp_'+year+'_*.nc'
    datasets = []
    for subfolder in frc_sub_paths:
        file_list = file_list+glob.glob(frc_path+subfolder+file_name_format)
    file_list = sorted(file_list)
    for file in file_list:
        print('loading ',file)
        theta_full = xr.open_dataset(file,decode_times=True)
        theta_full = theta_full.assign_coords({'xt_ocean': (theta_full['xt_ocean'] + 360) % 360}).sortby('xt_ocean')
        theta = theta_full.sel(yt_ocean=slice(*grd_lat_ax),xt_ocean=slice(*grd_lon_ax))
        theta_full.close()
        theta.load()
        datasets.append(theta)
        del theta
    theta = xr.concat(datasets, dim='Time')
    print('done, size ',theta.nbytes/1e9,'G')

    file_list=[]
    file_name_format = 'ocean_salt_'+year+'_*.nc'
    datasets = []
    for subfolder in frc_sub_paths:
        file_list = file_list+glob.glob(frc_path+subfolder+file_name_format)
    file_list = sorted(file_list)
    for file in file_list:
        print('loading ',file)
        temp = xr.open_dataset(file,decode_times=True)
        temp = temp.assign_coords({'xt_ocean': (temp['xt_ocean'] + 360) % 360}).sortby('xt_ocean')
        temp = temp.sel(yt_ocean=slice(*grd_lat_ax),xt_ocean=slice(*grd_lon_ax))
        temp.load()
        datasets.append(temp)
        del temp
    salt = xr.concat(datasets, dim='Time')
    print('done, size ',salt.nbytes/1e9,'G')

    file_list=[]
    file_name_format = 'ocean_u_'+year+'_*.nc'
    datasets = []
    for subfolder in frc_sub_paths:
        file_list = file_list+glob.glob(frc_path+subfolder+file_name_format)
    file_list = sorted(file_list)
    for file in file_list:
        print('loading ',file)
        temp = xr.open_dataset(file,decode_times=True)
        temp = temp.assign_coords({'xu_ocean': (temp['xu_ocean'] + 360) % 360}).sortby('xu_ocean')
        temp = temp.sel(yu_ocean=slice(*grd_lat_ax),xu_ocean=slice(*grd_lon_ax))
        temp.load()
        datasets.append(temp)
        del temp
    uvel = xr.concat(datasets, dim='Time')
    print('done, size ',uvel.nbytes/1e9,'G')

    file_list=[]
    file_name_format = 'ocean_v_'+year+'_*.nc'
    datasets = []
    for subfolder in frc_sub_paths:
        file_list = file_list+glob.glob(frc_path+subfolder+file_name_format)
    file_list = sorted(file_list)
    for file in file_list:
        print('loading ',file)
        temp = xr.open_dataset(file,decode_times=True)
        temp = temp.assign_coords({'xu_ocean': (temp['xu_ocean'] + 360) % 360}).sortby('xu_ocean')
        temp = temp.sel(yu_ocean=slice(*grd_lat_ax),xu_ocean=slice(*grd_lon_ax))
        temp.load()
        datasets.append(temp)
        del temp
    vvel = xr.concat(datasets, dim='Time')
    print('done, size ',vvel.nbytes/1e9,'G')

    file_list=[]
    file_name_format = 'ocean_eta_t_'+year+'_*.nc'
    datasets = []
    for subfolder in frc_sub_paths:
        file_list = file_list+glob.glob(frc_path+subfolder+file_name_format)
    file_list = sorted(file_list)
    for file in file_list:
        print('loading ',file)
        temp = xr.open_dataset(file,decode_times=True)
        temp = temp.assign_coords({'xt_ocean': (temp['xt_ocean'] + 360) % 360}).sortby('xt_ocean')
        temp = temp.sel(yt_ocean=slice(*grd_lat_ax),xt_ocean=slice(*grd_lon_ax))
        temp.load()
        datasets.append(temp)
        del temp
    zeta = xr.concat(datasets, dim='Time')
    print('done, size ',zeta.nbytes/1e9,'G')




    if (year=='2008') | (year=='2012') | (year=='2016') | (year=='2020') | (year=='2004'): #bit of a hack
        try:
            print('looking for whether leap day is present')
            print(theta.sel(Time=year+'-02-29T12:00'))
        except:
            print('that day isn''t present')
        else:
            print('remove leap year data because no one needs that sht')        
            theta = theta.drop_sel(Time=year+'-02-29T12:00')
            salt = salt.drop_sel(Time=year+'-02-29T12:00')
            uvel = uvel.drop_sel(Time=year+'-02-29T12:00')
            vvel = vvel.drop_sel(Time=year+'-02-29T12:00')
            zeta = zeta.drop_sel(Time=year+'-02-29T12:00')


    print('now we have our time coordinate sorted, prepare the time parameters for daily forcing')
    time_length = len(theta.Time)
    time = np.arange(time_length/time_length/2,time_length+time_length/time_length/2,time_length/time_length) #time values for each data point (units of days)
    cycle= 365.0 #days over which to cycle the forcing data
    print(' make array in preparation, with time length ')

    salt_i = np.full((*salt.Time.shape,*LonInterpSurface.shape),np.NaN)
    theta_i = np.full((*theta.Time.shape,*LonInterpSurface.shape),np.NaN)
    uvel_i = np.full((*uvel.Time.shape,*LonInterpSurface.shape),np.NaN)
    vvel_i = np.full((*vvel.Time.shape,*LonInterpSurface.shape),np.NaN)
    zeta_i = np.full((*zeta.Time.shape,*LonInterpSurface2d.shape),np.NaN)
        
    
    print(' start time loop and interpolate values to new locations')
    from scipy.interpolate import RegularGridInterpolator
    for tt in range(len(salt.Time)):
        print('processing time value: ',tt)
        # make the interpolator for each time point
        interp_function_salt = RegularGridInterpolator((salt.xt_ocean.values, salt.yt_ocean.values, salt.st_ocean.values ), np.transpose(salt.salt.isel(Time=tt).values,(2,1,0)), method='linear', bounds_error=False, fill_value=np.NaN)
        interp_function_theta = RegularGridInterpolator((theta.xt_ocean.values, theta.yt_ocean.values, theta.st_ocean.values ), np.transpose(theta.temp.isel(Time=tt).values,(2,1,0)), method='linear', bounds_error=False, fill_value=np.NaN)
        interp_function_uvel = RegularGridInterpolator((uvel.xu_ocean.values, uvel.yu_ocean.values, uvel.st_ocean.values ), np.transpose(uvel.u.isel(Time=tt).values,(2,1,0)), method='linear', bounds_error=False, fill_value=np.NaN)
        interp_function_vvel = RegularGridInterpolator((vvel.xu_ocean.values, vvel.yu_ocean.values, vvel.st_ocean.values ), np.transpose(vvel.v.isel(Time=tt).values,(2,1,0)), method='linear', bounds_error=False, fill_value=np.NaN)
        interp_function_zeta = RegularGridInterpolator((zeta.xt_ocean.values, zeta.yt_ocean.values), np.transpose(zeta.eta_t.isel(Time=tt).values,(1,0)), method='linear', bounds_error=False, fill_value=np.NaN)

        # interpolate
        salt_i[tt,:,:] = interp_function_salt((LonInterpSurface, LatInterpSurface, -DepthsInterpSurface))
        theta_i[tt,:,:] = interp_function_theta((LonInterpSurface, LatInterpSurface, -DepthsInterpSurface))
        uvel_i[tt,:,:] = interp_function_uvel((LonInterpSurface, LatInterpSurface, -DepthsInterpSurface))
        vvel_i[tt,:,:] = interp_function_vvel((LonInterpSurface, LatInterpSurface, -DepthsInterpSurface))
        zeta_i[tt,:] = interp_function_zeta((LonInterpSurface2d, LatInterpSurface2d))


        salt_i[tt,:,:] = inpaint_nans(salt_i[tt,:,:])
        theta_i[tt,:,:] = inpaint_nans(theta_i[tt,:,:])
        uvel_i[tt,:,:] = inpaint_nans(uvel_i[tt,:,:])
        vvel_i[tt,:,:] = inpaint_nans(vvel_i[tt,:,:])    
        zeta_i[tt,:] = fill_nans_scipy1(zeta_i[tt,:])    

    ##/ NO CONVERSION FROM CONS TEMP/ABS SALT REQUIRED WITH BRAN.

    # print('convert conservative temperature to potential temperature; and absolute salinity to prac salinity.')
    # import gsw 

    # salt_abs_i = salt_i.copy()

    # theta_cons_i = theta_i.copy()

    # theta_i = gsw.pt_from_CT(salt_i,theta_i-273.15)
    # salt_i = gsw.SP_from_SA(salt_i, DepthsInterpSurface, LonInterpSurface, LatInterpSurface)
    ##/
    
    ##/ DO YOU WANT ZERO ZETA??
    # make the zeta variable. For now set it to zeros
    # zeta_i = np.zeros_like(salt_i[:,0,:])
    ##/
    
    print(' separate out theta, salt, uvvel, vvel, zeta into each direction')

    # left or "WEST"
    # SET TO 0 FOR NOW - TO UPDATE IN time.
    select=slice(0,z_rho_3d.shape[1])
    salt_west = salt_i[:,:,select]
    temp_west = theta_i[:,:,select]
    u_west = uvel_i[:,:,select]
    v_west = vvel_i[:,:,select]
    # ubar_west = np.zeros(uvel_i[:,:,select].shape)
    # vbar_west = np.zeros(vvel_i[:,:,select].shape)
    zeta_west = zeta_i[:,select]

    # top or "NORTH"
    select=slice(z_rho_3d.shape[1],z_rho_3d.shape[1]+z_rho_3d.shape[2])
    salt_north = salt_i[:,:,select]
    temp_north = theta_i[:,:,select]
    u_north = uvel_i[:,:,select]
    v_north = vvel_i[:,:,select]
    # ubar_north = np.zeros(uvel_i[:,:,select].shape)
    # vbar_north = np.zeros(vvel_i[:,:,select].shape)
    zeta_north = zeta_i[:,select]

    # right or "EAST"
    select=slice(z_rho_3d.shape[1]+z_rho_3d.shape[2],z_rho_3d.shape[1]+z_rho_3d.shape[2]+z_rho_3d.shape[1])
    temp_east = theta_i[:,:,select]
    salt_east = salt_i[:,:,select]
    u_east = uvel_i[:,:,select]
    v_east = vvel_i[:,:,select]
    # ubar_east = np.zeros(uvel_i[:,:,select].shape)
    # vbar_east = np.zeros(vvel_i[:,:,select].shape)
    zeta_east = zeta_i[:,select]

    # bottom or "SOUTH"
    select=slice(z_rho_3d.shape[1]+z_rho_3d.shape[2]+z_rho_3d.shape[1],z_rho_3d.shape[1]+z_rho_3d.shape[2]+z_rho_3d.shape[1]+z_rho_3d.shape[2])
    temp_south = theta_i[:,:,select]
    salt_south = salt_i[:,:,select]
    u_south = uvel_i[:,:,select]
    v_south = vvel_i[:,:,select]
    # ubar_south = np.zeros(uvel_i[:,:,select].shape)
    # vbar_south = np.zeros(vvel_i[:,:,select].shape)
    zeta_south = zeta_i[:,select]

    print(' Now rotate angles!')
    angle3 = np.tile(grd.angle, (len(salt.Time), N,1,1))

    angle_south = angle3[:,:,0,:]
    u_south_unrot = u_south.copy()
    v_south_unrot = v_south.copy()
    u_south_latlon =  u_south.copy()
    v_south_latlon =  v_south.copy()
    uv_south = (u_south_latlon + 1j*v_south_latlon) * np.exp(-1j*angle_south)
    u_south_rho = uv_south.real
    v_south = uv_south.imag
    u_south = 0.5 * (u_south_rho[:,:,0:-1]+u_south_rho[:,:,1:])

    angle_east = angle3[:,:,:,-1]
    u_east_unrot = u_east.copy()
    v_east_unrot = v_east.copy()
    u_east_latlon =  u_east.copy()
    v_east_latlon =  v_east.copy()
    uv_east = (u_east_latlon + 1j*v_east_latlon) * np.exp(-1j*angle_east)
    u_east = uv_east.real
    v_east_rho = uv_east.imag
    v_east = 0.5 * (v_east_rho[:,:,0:-1]+v_east_rho[:,:,1:])

    angle_north = angle3[:,:,-1,:]
    u_north_unrot = u_north.copy()
    v_north_unrot = v_north.copy()
    u_north_latlon =  u_north.copy()
    v_north_latlon =  v_north.copy()
    uv_north = (u_north_latlon + 1j*v_north_latlon) * np.exp(-1j*angle_north)
    u_north_rho = uv_north.real
    v_north = uv_north.imag
    u_north = 0.5 * (u_north_rho[:,:,0:-1]+u_north_rho[:,:,1:])

    angle_west = angle3[:,:,:,0]
    u_west_unrot = u_west.copy()
    v_west_unrot = v_west.copy()
    u_west_latlon =  u_west.copy()
    v_west_latlon =  v_west.copy()
    uv_west = (u_west_latlon + 1j*v_west_latlon) * np.exp(-1j*angle_west)
    u_west = uv_west.real
    v_west_rho = uv_west.imag
    v_west = 0.5 * (v_west_rho[:,:,0:-1]+v_west_rho[:,:,1:])

    print('calculate mapping from u to ubar')
    z_rho,z_w,=GridStiffness.calc_z(Vtransform,Vstretching,theta_s,theta_b,Tcline,hc,N,grd.h,np.zeros(grd.h.shape)) # extract the z_w values

    # compute depth average velocity ubar and vbar
    # get z at the right position
    # based on: https://github.com/ESMG/pyroms/blob/5ea501ef904b01036dd2a0909b7bdc61a56e7eff/examples/Arctic_SODA3.3.1/remap_bdry_uv.py#L309
    z_u_north = 0.5 * (z_w[:,-1,:-1] + z_w[:,-1,1:]) # TOP: looks like my _east
    z_v_north = 0.5 * (z_w[:,-1,:] + z_w[:,-2,:])
    z_u_south = 0.5 * (z_w[:,0,:-1] + z_w[:,0,1:]) # BOTTOM: looks like my _west
    z_v_south = 0.5 * (z_w[:,0,:] + z_w[:,1,:])
    z_u_east = 0.5 * (z_w[:,:,-1] + z_w[:,:,-2]) # RIGHT: looks like my _north
    z_v_east = 0.5 * (z_w[:,:-1,-1] + z_w[:,1:,-1])
    z_u_west = 0.5 * (z_w[:,:,0] + z_w[:,:,1]) # LEFT: looks like my _south
    z_v_west = 0.5 * (z_w[:,:-1,0] + z_w[:,1:,0])
    # based on the averaging, this looks like pyroms expects:
    #        _north
    # 
    # _west         _east
    # 
    #        _south 

    ubar_north = np.zeros(u_north[:,0,:].shape)
    ubar_south = np.zeros(u_south[:,0,:].shape)
    ubar_east = np.zeros(u_east[:,0,:].shape)
    ubar_west = np.zeros(u_west[:,0,:].shape)
    vbar_north = np.zeros(v_north[:,0,:].shape)
    vbar_south = np.zeros(v_south[:,0,:].shape)
    vbar_east = np.zeros(v_east[:,0,:].shape)
    vbar_west = np.zeros(v_west[:,0,:].shape)

    for tt in range(len(salt.Time)):
        print('processing time value: ',tt)

        for i in range(u_north[tt,:,:].shape[1]):
            ubar_north[tt,i] = (u_north[tt,:,i] * np.diff(z_u_north[:,i])).sum() / -z_u_north[0,i]
            ubar_south[tt,i] = (u_south[tt,:,i] * np.diff(z_u_south[:,i])).sum() / -z_u_south[0,i]
        for i in range(v_north[tt,:,:].shape[1]):
            vbar_north[tt,i] = (v_north[tt,:,i] * np.diff(z_v_north[:,i])).sum() / -z_v_north[0,i]
            vbar_south[tt,i] = (v_south[tt,:,i] * np.diff(z_v_south[:,i])).sum() / -z_v_south[0,i]
        for j in range(u_east[tt,:,:].shape[1]):
            ubar_east[tt,j] = (u_east[tt,:,j] * np.diff(z_u_east[:,j])).sum() / -z_u_east[0,j]
            ubar_west[tt,j] = (u_west[tt,:,j] * np.diff(z_u_west[:,j])).sum() / -z_u_west[0,j]
        for j in range(v_east[tt,:,:].shape[1]):
            vbar_east[tt,j] = (v_east[tt,:,j] * np.diff(z_v_east[:,j])).sum() / -z_v_east[0,j]
            vbar_west[tt,j] = (v_west[tt,:,j] * np.diff(z_v_west[:,j])).sum() / -z_v_west[0,j]


    print(' check for nans and other bad values')
    variables = ['theta_s','theta_b','Tcline','hc','sc_r','Cs_r','Time']



    for arr_name in variables:
        arr = locals().get(arr_name)
        if arr is not None:
            if np.isnan(arr).any():
                print(f"Warning: NaN found in array '{arr_name}'")
            # else:
            #     print('no NaN in '+arr_name)

    #check for nans in all that boundary data

    boundary = ['_south','_east','_north','_west']
    variables = ['temp','salt','u','v','ubar','vbar']

    for bnd in boundary:
        for var in variables:
            arr_name = str(var+bnd)
            arr = locals().get(str(var+bnd))
            if arr is not None:
                if np.isnan(arr).any():
                    print(f"Warning: NaN found in array '{arr_name}'")
                # else:
                #     print('no NaN in '+arr_name)

    print('quick check of all field sizes')
    boundary = ['_south','_east','_north','_west']
    variables = ['temp','salt','u','v','ubar','vbar']

    for bnd in boundary:
        for var in variables:
            arr_name = str(var+bnd)
            arr = locals().get(str(var+bnd))
            print(f" Size of '{arr_name}' is {arr.shape}")


    print('make netcdf data, ready to be saved')

    temp_time=time.copy()
    salt_time=time.copy()
    v3d_time=time.copy()
    v2d_time=time.copy()
    zeta_time=time.copy()

    theta_s_da = xr.DataArray(theta_s,name='theta_s',attrs={'long_name': 'S-coordinate surface control parameter', 'units': 'nondimensional'} )
    theta_b_da = xr.DataArray(theta_s,name='theta_b',attrs={'long_name': 'S-coordinate bottom control parameter', 'units': 'nondimensional'} )
    tcline_da = xr.DataArray(Tcline,name='Tcline',attrs={'long_name': 'S-coordinate surface/bottom layer width', 'units': 'meter'} )
    hc_da = xr.DataArray(hc,name='hc',attrs={'long_name': 'S-coordinate parameter, critical depth', 'units': 'meter'} )
    sc_r_da = xr.DataArray(sc_r,name='sc_r',dims=['s_rho'],attrs={'long_name': 'S-coordinate at RHO-points', 'units': 'nondimensional'} )
    cs_r_da = xr.DataArray(Cs_r,name='Cs_r',dims=['s_rho'],attrs={'long_name': 'S-coordinate stretching curves at RHO-points', 'units': 'nondimensional','valid_min':-1.0,'valid_max':0.0} )
    temp_time_da = xr.DataArray(temp_time,name='temp_time',dims=['temp_time'],attrs={'long_name': 'time for temperature forcing', 'units': 'day','cycle_length':cycle} )
    salt_time_da = xr.DataArray(salt_time,name='salt_time',dims=['salt_time'],attrs={'long_name': 'time for salinity forcing', 'units': 'day','cycle_length':cycle} )
    v3d_time_da = xr.DataArray(v3d_time,name='v3d_time',dims=['v3d_time'],attrs={'long_name': 'time for 3d-velocity forcing', 'units': 'day','cycle_length':cycle} )
    v2d_time_da = xr.DataArray(v2d_time,name='v2d_time',dims=['v2d_time'],attrs={'long_name': 'time for 2d-velocity forcing', 'units': 'day','cycle_length':cycle} )
    zeta_time_da = xr.DataArray(zeta_time,name='zeta_time',dims=['zeta_time'],attrs={'long_name': 'time for ssh forcing', 'units': 'day','cycle_length':cycle} )

    # make nth boundary data
    temp_north_da = xr.DataArray(temp_north,name='temp_north',dims=['temp_time','s_rho', 'xi_rho'],attrs={'long_name': 'northern boundary potential temperature',
     'standard_name': 'temp_north','units': 'Celsius','_FillValue':'-1e34','missing_value':'-1e34'})
    salt_north_da = xr.DataArray(salt_north,name='salt_north',dims=['salt_time','s_rho', 'xi_rho'],attrs={'long_name': 'northern boundary salinity',
     'standard_name': 'temp_north','units': 'PSU','_FillValue':'-1e34','missing_value':'-1e34'})
    u_north_da = xr.DataArray(u_north,name='u_north',dims=['v3d_time','s_rho','xi_u'],attrs={'long_name': 'northern boundary u-momentum component',
     'standard_name': 'u_north','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    v_north_da = xr.DataArray(v_north,name='v_north',dims=['v3d_time','s_rho', 'xi_rho'],attrs={'long_name': 'northern boundary v-momentum component',
     'standard_name': 'v_north','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    ubar_north_da = xr.DataArray(ubar_north,name='ubar_north',dims=['v2d_time','xi_u'],attrs={'long_name': 'northern boundary vertically integrated u-momentum component',
     'standard_name': 'ubar_north','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    vbar_north_da = xr.DataArray(vbar_north,name='vbar_north',dims=['v2d_time','xi_rho'],attrs={'long_name': 'northern boundary vertically integrated v-momentum component',
     'standard_name': 'vbar_north','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    zeta_north_da = xr.DataArray(zeta_north,name='zeta_north',dims=['zeta_time','xi_rho'],attrs={'long_name': 'northern boundary sea surface height',
     'standard_name': 'zeta_north','units': 'meter','_FillValue':'-1e34','missing_value':'-1e34'})

    # make sth boundary data
    temp_south_da = xr.DataArray(temp_south,name='temp_south',dims=['temp_time','s_rho','xi_rho'],attrs={'long_name': 'southern boundary potential temperature',
     'standard_name': 'temp_south','units': 'Celsius','_FillValue':'-1e34','missing_value':'-1e34'})
    salt_south_da = xr.DataArray(salt_south,name='salt_south',dims=['salt_time','s_rho','xi_rho'],attrs={'long_name': 'southern boundary salinity',
     'standard_name': 'temp_south','units': 'PSU','_FillValue':'-1e34','missing_value':'-1e34'})
    u_south_da = xr.DataArray(u_south,name='u_south',dims=['v3d_time','s_rho','xi_u'],attrs={'long_name': 'southern boundary u-momentum component',
     'standard_name': 'u_south','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    v_south_da = xr.DataArray(v_south,name='v_south',dims=['v3d_time','s_rho', 'xi_rho'],attrs={'long_name': 'southern boundary v-momentum component',
     'standard_name': 'v_south','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    ubar_south_da = xr.DataArray(ubar_south,name='ubar_south',dims=['v2d_time','xi_u'],attrs={'long_name': 'southern boundary vertically integrated u-momentum component',
     'standard_name': 'ubar_south','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    vbar_south_da = xr.DataArray(vbar_south,name='vbar_south',dims=['v2d_time','xi_rho'],attrs={'long_name': 'southern boundary vertically integrated v-momentum component',
     'standard_name': 'vbar_south','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    zeta_south_da = xr.DataArray(zeta_south,name='zeta_south',dims=['zeta_time','xi_rho'],attrs={'long_name': 'southern boundary sea surface height',
     'standard_name': 'zeta_south','units': 'meter','_FillValue':'-1e34','missing_value':'-1e34'})

    # make eastern boundary data
    temp_east_da = xr.DataArray(temp_east,name='temp_east',dims=['temp_time','s_rho','eta_rho'],attrs={'long_name': 'eastern boundary potential temperature',
     'standard_name': 'temp_east','units': 'Celsius','_FillValue':'-1e34','missing_value':'-1e34'})
    salt_east_da = xr.DataArray(salt_east,name='salt_east',dims=['salt_time','s_rho', 'eta_rho'],attrs={'long_name': 'eastern boundary salinity',
     'standard_name': 'temp_east','units': 'PSU','_FillValue':'-1e34','missing_value':'-1e34'})
    u_east_da = xr.DataArray(u_east,name='u_east',dims=['v3d_time','s_rho', 'eta_rho'],attrs={'long_name': 'eastern boundary u-momentum component',
     'standard_name': 'u_east','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    v_east_da = xr.DataArray(v_east,name='v_east',dims=['v3d_time','s_rho', 'eta_v'],attrs={'long_name': 'eastern boundary v-momentum component',
     'standard_name': 'v_east','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    ubar_east_da = xr.DataArray(ubar_east,name='ubar_east',dims=['v2d_time','eta_rho'],attrs={'long_name': 'eastern boundary vertically integrated u-momentum component',
     'standard_name': 'ubar_east','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    vbar_east_da = xr.DataArray(vbar_east,name='vbar_east',dims=['v2d_time','eta_v'],attrs={'long_name': 'eastern boundary vertically integrated v-momentum component',
     'standard_name': 'vbar_east','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    zeta_east_da = xr.DataArray(zeta_east,name='zeta_east',dims=['zeta_time','eta_rho'],attrs={'long_name': 'eastern boundary sea surface height',
     'standard_name': 'zeta_east','units': 'meter','_FillValue':'-1e34','missing_value':'-1e34'})

    # make western boundary data
    temp_west_da = xr.DataArray(temp_west,name='temp_west',dims=['temp_time','s_rho', 'eta_rho'],attrs={'long_name': 'western boundary potential temperature',
     'standard_name': 'temp_west','units': 'Celsius','_FillValue':'-1e34','missing_value':'-1e34'})
    salt_west_da = xr.DataArray(salt_west,name='salt_west',dims=['salt_time','s_rho', 'eta_rho'],attrs={'long_name': 'western boundary salinity',
     'standard_name': 'temp_west','units': 'PSU','_FillValue':'-1e34','missing_value':'-1e34'})
    u_west_da = xr.DataArray(u_west,name='u_west',dims=['v3d_time','s_rho', 'eta_rho'],attrs={'long_name': 'western boundary u-momentum component',
     'standard_name': 'u_west','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    v_west_da = xr.DataArray(v_west,name='v_west',dims=['v3d_time','s_rho', 'eta_v'],attrs={'long_name': 'western boundary v-momentum component',
     'standard_name': 'v_west','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    ubar_west_da = xr.DataArray(ubar_west,name='ubar_west',dims=['v2d_time','eta_rho'],attrs={'long_name': 'western boundary vertically integrated u-momentum component',
     'standard_name': 'ubar_west','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    vbar_west_da = xr.DataArray(vbar_west,name='vbar_west',dims=[ 'v2d_time','eta_v'],attrs={'long_name': 'western boundary vertically integrated v-momentum component',
     'standard_name': 'vbar_west','units': 'meter second-1','_FillValue':'-1e34','missing_value':'-1e34'})
    zeta_west_da = xr.DataArray(zeta_west,name='zeta_west',dims=[ 'zeta_time','eta_rho'],attrs={'long_name': 'western boundary sea surface height',
     'standard_name': 'zeta_west','units': 'meter','_FillValue':'-1e34','missing_value':'-1e34'})

    frc = xr.Dataset({'theta_s':theta_s_da,
                    'theta_b':theta_b_da,
                    'Tcline':tcline_da,
                    'hc':hc_da,
                    'sc_r':sc_r_da,
                    'Cs_r':cs_r_da,
                    'temp_time':temp_time_da,
                    'salt_time':salt_time_da,
                    'v3d_time':v3d_time_da,
                    'v2d_time':v2d_time_da,
                    'zeta_time':zeta_time_da,
                    'temp_north':temp_north_da,
                    'salt_north':salt_north_da,
                    'u_north':u_north_da,
                    'v_north':v_north_da,
                    'ubar_north':ubar_north_da,
                    'vbar_north':vbar_north_da,
                    'zeta_north':zeta_north_da,
                    'temp_south':temp_south_da,
                    'salt_south':salt_south_da,
                    'u_south':u_south_da,
                    'v_south':v_south_da,
                    'ubar_south':ubar_south_da,
                    'vbar_south':vbar_south_da,
                    'zeta_south':zeta_south_da,
                    'temp_east':temp_east_da,
                    'salt_east':salt_east_da,
                    'u_east':u_east_da,
                    'v_east':v_east_da,
                    'ubar_east':ubar_east_da,
                    'vbar_east':vbar_east_da,
                    'zeta_east':zeta_east_da,              
                    'temp_west':temp_west_da,
                    'salt_west':salt_west_da,
                    'u_west':u_west_da,
                    'v_west':v_west_da,
                    'ubar_west':ubar_west_da,
                    'vbar_west':vbar_west_da,
                    'zeta_west':zeta_west_da},
                   attrs={'title':'ROMS Lateral Boundaries',
                          'history': 'BOUNDARY file using make_bry.py,'+str(datetime.date.today()),
                          'clim_name':out_file,
                          'grd_file':grd_file,
                          'type': 'BOUNDARY file'})

    print('saving to ',out_file)
    frc.to_netcdf(out_file)

making forcing file for year  2012
loading data from  ['./daily/']
set settings for each file
out file name is:  /g/data/jk72/deg581/se-qld-setup/data/proc/seqld_1km_daily_2012_bry.nc
start loading data
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_01.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_02.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_03.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_04.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_05.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_06.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_07.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_08.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_09.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_10.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_11.nc
loading  /g/data/gb6/BRAN/BRAN2020/./daily/ocean_temp_2012_12.nc
done, size  3.319